In [1]:
import pandas as pd
import numpy as np
from glob import glob

In [2]:
# INPUTS------------------------------------------------------------------
# Number of Components
k = 4
# Component names
variable = ['angle1','angle2','angle3','angle4']

# layer names
# Used for extracting corresponding layer size for design while reading
# the des file
layer_name = ['m[0]','m[1]','m[2]','m[3]']

# Max number of layers
max_len = 20
#----- END INPUTS---------------------------------------------------------

In [10]:
files = glob("./redundancy_check_for_existing_runs/*.des")

df_red = pd.DataFrame(columns=['File Name','Logic Repeated(% of actual runs)','Logic Repeated(% of total)',
                               'Conventional Repeated(% of total)','Overall Repeated(% of total)'])
df_red['File Name'] = pd.Series(files)

def get_label_line(des_file):
    f = open(des_file,'rb')
    lines = f.readlines()
    line_end = len(lines)
    
    start_table = 0
    
    # Find the line with variable names
    for i in range(len(lines)):
        line_tmp = lines[i].split()

        try:
            if line_tmp[0]=='<ID>':
                label_line = line_tmp
                start_table = i+1
                break
        except:
            pass
        
    return label_line,start_table
        

# Function to collect final component layers
def get_component_layers(key,n,line,pos):
    comp = line[pos[key]:pos[key]+n[key]]
    comp = [str(int(float(k))) for k in comp]
    comp = "".join(comp)
    return comp

def get_layer(n,line,pos,variable=variable):
    # Get component final layers
    comps = [get_component_layers(x,n,line,pos) for x in variable]
    return comps
            
    
for des_file in files:
    
    df1 = pd.DataFrame(columns=variable)
    
    label_line,start_table = get_label_line(des_file)
    
    f = open(des_file,'rb')
    lines = f.readlines()
    line_end = len(lines)
    
    # Create a position dictionary for starting point of the component layer
    start_pos = [label_line.index(x + '[0]') for x in variable]
    pos_dict = dict(zip(variable,start_pos))
    
    
    # Find the line with variable names
    for i in range(start_table,line_end):
        line = lines[i].split()

        # Create a layer_size position dict
        layer_size = [int(float(line[label_line.index(x)])) for x in layer_name]
        layer_dict = dict(zip(variable,layer_size))
        
        layer = get_layer(layer_dict,line,pos_dict)
        
        df_tmp = pd.DataFrame([layer],columns=variable)
        df1 = df1.append(df_tmp,ignore_index=True)
        
    rep = df1[df1.duplicated(keep='first')].shape[0]
    rep_perc = rep*100./(df1.shape[0])
    df_red['Logic Repeated(% of actual runs)'][df_red['File Name'] == des_file] = rep_perc
    df_red['Logic Repeated(% of total)'][df_red['File Name'] == des_file] = rep*100/4800.
    
    conv_rep = 4800 - df1.shape[0]
    df_red['Conventional Repeated(% of total)'][df_red['File Name']== des_file] = conv_rep*100/4800.
    
    overall_rep_perc = (conv_rep + rep)*100/4800. 
    df_red['Overall Repeated(% of total)'][df_red['File Name']== des_file] = overall_rep_perc 
        
    print("Completed:    %s"%des_file)    
    
df_red.to_csv("redundancy_check_for_existing_runs.csv",index=False)

Completed:    ./redundancy_check_for_existing_runs\moga-generation-based-seed1.des
Completed:    ./redundancy_check_for_existing_runs\moga-generation-based-seed2.des
Completed:    ./redundancy_check_for_existing_runs\moga-generation-based-seed3.des
Completed:    ./redundancy_check_for_existing_runs\moga-generation-based-seed4.des
Completed:    ./redundancy_check_for_existing_runs\moga-generation-based-seed5.des
Completed:    ./redundancy_check_for_existing_runs\moga-steadystate-seed1.des
Completed:    ./redundancy_check_for_existing_runs\moga-steadystate-seed2.des
Completed:    ./redundancy_check_for_existing_runs\moga-steadystate-seed3.des
Completed:    ./redundancy_check_for_existing_runs\moga-steadystate-seed4.des
Completed:    ./redundancy_check_for_existing_runs\moga-steadystate-seed5.des
Completed:    ./redundancy_check_for_existing_runs\nsga-generation-based-seed1.des
Completed:    ./redundancy_check_for_existing_runs\nsga-generation-based-seed2.des
Completed:    ./redundancy_ch

In [32]:
cc = ['Logic Repeated(% of total)','Conventional Repeated(% of total)','Overall Repeated(% of total)']

In [19]:
df_red = pd.read_csv("./redundancy_check_for_existing_runs.csv")

In [33]:
df_red.loc[10:14]

,File Name,Logic Repeated(% of actual runs),Logic Repeated(% of total),Conventional Repeated(% of total),Overall Repeated(% of total)
10,./redundancy_check_for_existing_runs\nsga-gene...,14.650538,9.083333,38.000000,47.083333
11,./redundancy_check_for_existing_runs\nsga-gene...,24.187832,17.062500,29.458333,46.520833
12,./redundancy_check_for_existing_runs\nsga-gene...,17.007673,11.083333,34.833333,45.916667
13,./redundancy_check_for_existing_runs\nsga-gene...,19.599109,12.833333,34.520833,47.354167
14,./redundancy_check_for_existing_runs\nsga-gene...,12.563613,8.229167,34.500000,42.729167
